# Master API Nptebook
* Functions for downloading data using APIs from myfantasyleague.com for fantasy football data
* Brock Ricker
* https://github.com/brock-ricker
* Created 07/01/2022
* v1.1: 07/09/2022 - added year to points allowed, and year/week to projected score

In [159]:
#import modules here
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine
import time
import logging

In [160]:
#Setup Variables

#define user agent for my fantasy API use
agent = "brockr1"
#my Fantasy League, League ID
league_id = 20896
#create db name - this will be SQLite
db_name = "test"
#Years of league history
years = [2017,2018,2019,2020,2021]

In [161]:
# create sqlite engine for a db using name defined above
engine = create_engine(f"sqlite:///{db_name}.db", echo=False)
#create connection to the engine
conn = engine.connect()

In [ ]:
#setup logging info
#naming log based on date/time
t=time.localtime()
date = time.strftime("%y%m%d_%H%M%S", t)
#logging settings
logging.basicConfig(filename=f"{date}.log", encoding='utf-8', level=logging.INFO)
#initialize errors variable = False
errors = False

# Global Functions

In [280]:
#defining pause function (for pause time between yearly API pulls)
def pause(t=2):
    print("pausing to prevent lockout of API")
    time.sleep(time)

In [302]:
#defining pause function (for pause time between weekly API pulls)
def short_pause():
    time.sleep(1)

# Weekly Results
* This section covers functions from Weekly Results API requests

In [68]:
#define function for cleaning players table
def df_players_cleaner(df_players):

    df_players = df_players.astype({
        "id": int,
        "shouldStart": int,
        "score": float
    })

    df_players = df_players.rename(columns = {"shouldStart": "should_start"})

    return df_players

In [64]:
#define function for cleaning df_teams
def df_team_cleaner(df_teams):
    
    df_teams.drop(columns = ["player","comments"],inplace=True)
    
    df_teams = df_teams.astype({
        "score": float,
        "isHome": int,
        "id": int,
    })

    df_teams = df_teams.rename(columns = {"isHome": "is_home"})

    return df_teams

In [65]:
#define function for creating matchups table - !must run df_team_cleaner first!
def matchup_table_generator(df_teams):
    
    total_teams = len(df_teams)
    team1 = list(range(0,total_teams-1,2))
    team2 = list(range(1,total_teams,2))

    df_matchups = pd.DataFrame({
        "team1_id":list(df_teams.iloc[team1]["id"]),
        "team1_score":list(df_teams.iloc[team1]["score"]),
        "team2_id":list(df_teams.iloc[team2]["id"]),
        "team2_score":list(df_teams.iloc[team2]["score"])
    })

    df_matchups['winning_team'] = np.select([df_matchups['team1_score'] > df_matchups['team2_score'], df_matchups['team1_score'] < df_matchups['team2_score']], [df_matchups["team1_id"],df_matchups["team2_id"]], default="error")

    return df_matchups

In [251]:
#pulls weekly score data and saves to SQL server
#creates following tables: weekly_players, weekly_teams, weekly_matchup
def weekly_score_pull(years,agent,conn,league_id):
    logging.info("----------------------------------------weekly_score_pull----------------------------------------")
    i = 1
    for year in years:
        if i>1:
            pause()
        
        logging.info(f"begin parsing year: {year}")

        request_url = f"https://www54.myfantasyleague.com/{year}/export?L={league_id}&W=YTD&TYPE=weeklyResults&JSON=1&{agent}"
        req = requests.get(request_url)
        logging.info(f"connection results: {req}")
        req_data = req.json()
            
        weekly_results = req_data['allWeeklyResults']['weeklyResults']

        for single_week in weekly_results:
            if "matchup" not in single_week:
                break
            df_players = pd.json_normalize(single_week["matchup"],record_path=["franchise","player"])
            df_teams = pd.json_normalize(single_week["matchup"],record_path=["franchise"])
            
            #cleaning tables
            df_players = df_players_cleaner(df_players)
            df_teams = df_team_cleaner(df_teams)
            df_matchups = matchup_table_generator(df_teams)

            #adding weeks, years, regular_season to data_tables
            df_players["week"] = int(single_week["week"])
            df_players["year"] = year
            df_players["regular_season"] = int(single_week["matchup"][0]["regularSeason"])
            
            df_teams["week"] = int(single_week["week"])
            df_teams["year"] = year
            df_teams["regular_season"] = int(single_week["matchup"][0]["regularSeason"])
            
            df_matchups["week"] = int(single_week["week"])
            df_matchups["year"] = year
            df_matchups["regular_season"] = int(single_week["matchup"][0]["regularSeason"])        

            #writting to SQL
            df_players.to_sql("weekly_players", conn, if_exists="append");
            df_teams.to_sql("weekly_teams", conn, if_exists="append");
            df_matchups.to_sql("weekly_matchup", conn, if_exists="append");
            week = single_week["week"]
            logging.info(f"finished year: {year}, week: {week}")
        
        i = i+1

In [ ]:
#call function here
weekly_score_pull(years,agent,conn)

# Players
* This section covers functions from players

In [93]:
#cleans simple player data 
def clean_simple_players(players):
    players = players.astype({"id":int})
    players.rename(columns = {"status":"rookie"},inplace = True)
    players["rookie"].replace({"R":"yes"},inplace = True)
    players["rookie"].fillna("No", inplace= True)
    return players

In [117]:
#pulls player data and saves to SQL server
#creates following table: simple_players
def simple_player_pull (years,agent,conn,league_id):
    logging.info("----------------------------------------simple_player_pull----------------------------------------")
    i = 1
    for year in years:
        if i>1:
            pause()
            
        logging.info(f"begin parsing year: {year}")
        request_url = f"https://www54.myfantasyleague.com/{year}/export?L={league_id}&TYPE=players&JSON=1&{agent}"
        req = requests.get(request_url)
        req_data = req.json()
        logging.info(f"connection results: {req}")
        
        players = pd.json_normalize(req_data["players"],record_path=["player"])
        
        players = clean_simple_players(players)
        players["year"] = year
        
        players.to_sql("simple_players", conn, if_exists="append")
        i =i+1

In [118]:
#call function here
simple_player_pull(years,agent,conn)

begin parsing year: 2017
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2018
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2019
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2020
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2021
connection results: <Response [200]>


# ADP
* this section covers pulls from the ADP API (rookie and full)

In [146]:
#cleans ADP data
def clean_adp(adp):
    adp = adp.astype({
        'minPick':int, 
        'maxPick':int, 
        'draftsSelectedIn':int, 
        'id':int, 
        'averagePick':float, 
        'rank':int,
        'draftSelPct':int
    })
    return adp

In [155]:
#pulls adp data and saves to SQL server
#creates following table: adp
def adp_pull (years,agent,conn):
    logging.info("----------------------------------------adp_pull----------------------------------------")
    i = 1
    for year in years:
        if i>1:
            pause()

        logging.info(f"begin parsing year: {year}")
        request_url = f"https://api.myfantasyleague.com/{year}/export?TYPE=adp&JSON=1&{agent}&FCOUNT=12&&IS_KEEPER=N"
        req = requests.get(request_url)
        req_data = req.json()
        logging.info(f"connection results: {req}")
        
        adp = pd.json_normalize(req_data["adp"],record_path=["player"])

        adp = clean_adp(adp)
        
        adp["year"] = year
        
        adp.to_sql("adp", conn, if_exists="append")
        i =i+1

In [164]:
#This function does not work yet due to missing data no the website
def rookie_adp_pull (years,agent=agent,conn=conn):
    rookie_adp = pd.DataFrame()
    i = 1
    for year in years:
        if i>1:
            pause()

        print("begin parsing year:",year)
        request_url = f"https://api.myfantasyleague.com/{year}/export?TYPE=adp&JSON=1&{agent}&FCOUNT=12&&IS_KEEPER=R"
        req = requests.get(request_url)
        req_data = req.json()
        print("connection results:",req)
        
        rookie_adp = pd.json_normalize(req_data["adp"],record_path=["player"],errors="ignore")

        rookie_adp = clean_adp(rookie_adp)
        
        rookie_adp["year"] = year
        
        rookie_adp.to_sql("rookie_adp", conn, if_exists="append")
        i =i+1

In [157]:
#call function here
adp_pull(years)

begin parsing year: 2017
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2018
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2019
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2020
connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2021
connection results: <Response [200]>


In [ ]:
#missing 2018 data from website so this function is not called
#rookie_adp_pull(years)

# Points Allowed
* This section covers all the data from the points allowed API

In [246]:
#cleans points allowed table
def clean_points_allowed(points_allowed):
    
    #convert datatype
    points_allowed = points_allowed.astype({"points":float})
    #rename columns
    points_allowed.rename(columns = {"name":"pos"},inplace=True)
    #convert to points/week
    points_allowed["points"] = (points_allowed["points"]/16).round(2)
    #calculate total points and make into df
    total_points = pd.DataFrame({
        "points":[points_allowed["points"].sum()],
        "pos":["ALL"]})
    #add total points
    points_allowed = pd.concat([points_allowed,total_points],ignore_index=True)

    return points_allowed

In [275]:
#pulls points allowed data and saves to SQL server
#creates following table: points_allowed
def points_allowed_pull(years,agent,conn,league_id):
    logging.info("----------------------------------------points_allowed_pull----------------------------------------")
    i = 1
    for year in years:
        if i>1:
            pause()

        
        logging.info(f"begin parsing year: {year}")
        request_url = f"https://www54.myfantasyleague.com/{year}/export?L={league_id}&TYPE=pointsAllowed&JSON=1&{agent}"
        req = requests.get(request_url)
        logging.info(f"connection results: {req}")
        req_data = req.json()
            
        teams = req_data['pointsAllowed']["team"]

        for team in teams:
            points_allowed = pd.json_normalize(team,record_path=["position"])

            points_allowed = clean_points_allowed(points_allowed)

            points_allowed["team"] = team["id"]
            points_allowed["year"] = year

            points_allowed.to_sql("points_allowed", conn, if_exists="append")
        i = i+1

In [255]:
#call function here
points_allowed_pull(years)

begin parsing year: 2017
year 2017 connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2018
year 2018 connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2019
year 2019 connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2020
year 2020 connection results: <Response [200]>
pausing to prevent lockout of API
begin parsing year: 2021
year 2021 connection results: <Response [200]>


# Projected Scores
* This section covers all the data from the projected scores API

In [309]:
#cleans projected scores table
def clean_projected_scores(projected_scores):
    #delete rows with blank entries
    projected_scores = projected_scores.loc[~((projected_scores["score"]=="") | (projected_scores["id"]==""))]
    #convert datatypes
    projected_scores = projected_scores.astype({
    "score":float,
    "id":int
    })
    #rename columns
    projected_scores.rename(columns = {"score":"projected_scores"},inplace=True)
    return projected_scores

In [310]:
#pulls projected score data and saves to SQL server
#creates following table: projected_scores_pull
#very long pause due to numerous API requests
def projected_scores_pull(years,agent,conn,league_id):
    logging.info("----------------------------------------projected_scores_pull----------------------------------------")
    i = 1
    for year in years:
        if i>1:
            pause(300)
        
        logging.info(f"begin parsing year: {year}")
        weeks = list(range(1,19))
        
        for week in weeks:
            logging.info(f"parsing week: {week}")
            request_url = f"https://www54.myfantasyleague.com/{year}/export?L={league_id}&W={week}&TYPE=projectedScores&JSON=1&{agent}"
            req = requests.get(request_url)
            logging.info(f"connection results: {req}")
            req_data = req.json()

            projected_scores = pd.json_normalize(req_data["projectedScores"],record_path=["playerScore"])

            projected_scores = clean_projected_scores(projected_scores)

            projected_scores["week"] = week
            projected_scores["year"] = year

            projected_scores.to_sql("projected_scores", conn, if_exists="append")

            short_pause()


        i = i+1

In [ ]:
#call function here
#In order to avoid to API lockout, this fucntion takes ~5 minutes per year
projected_scores_pull(years)

# League Info
* covers all data from league info API

In [325]:
#cleans league data
def clean_league_teams(league_teams):
    league_teams = league_teams[["icon","name","id","logo"]]
    league_teams = league_teams.astype({"id":int})
    return league_teams

In [334]:
#pulls league data and saves to SQL server, this will replace any other entry in server
#creates following table: league_teams
def league_teams_pull(years,agent,conn,league_id):
    logging.info("----------------------------------------league_teams_pull----------------------------------------")
    year = years[-1]
    request_url = f"https://www54.myfantasyleague.com/{year}/export?L={league_id}&TYPE=league&JSON=1&{agent}"
    req = requests.get(request_url)
    logging.info(f"connection results: {req}")
    req_data =req.json()
    league_teams = pd.json_normalize(req_data["league"]["franchises"],record_path = ["franchise"])

    league_teams = clean_league_teams(league_teams)

    league_teams.to_sql("league_teams", conn, if_exists="replace")

In [333]:
#call function here, only 1 year is necesarry
league_teams_pull(2022)

begin parsing:
connection results: <Response [200]>
